In [1]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
import gradio as gr

/home/python/llm_projects/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv(override=True)
open_api_key = os.getenv("OPENAI_API_KEY")
if open_api_key:
    print(f'OPENAI_API_KEY: {open_api_key[:8]} exists')

OPENAI_API_KEY: sk-proj- exists


In [3]:
openai = OpenAI()
MODEL = "gpt-4o-mini"

In [4]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [5]:
ticket_price = {
    "usa": "BDT 20000",
    "canada": "BDT 25000",
    "uk": "BDT 30000",
    "australia": "BDT 35000"
}

def get_ticket_prices(destination):
    print(f"Ticket price for {destination} called")
    city = destination.lower()
    return ticket_price.get(city, "Sorry, we do not fly to that destination.")
get_ticket_prices("Canada")

Ticket price for Canada called


'BDT 25000'

In [6]:
# Tool definition for function calling
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [7]:
tools = [{"type": "function", "function": price_function}]

Response 
```json{
    "id": "chatcmpl-8IgndNLKqbrDT0jHxsuSA5t76uF0m",
    "object": "chat.completion",
    "created": 1699465117,
    "model": "gpt-3.5-turbo-0613",
    "choices": [
        {
            "index": 0,
            "message": {
                "role": "assistant",
                "content": null,
                "tool_calls": [
                    {
                        "id": "call_SDhXnJbvxSWwy1m1R1J43EmQ",
                        "type": "function",
                        "function": {
                            "name": "getLocation",
                            "arguments": "{}"
                        }
                    }
                ]
            },
            "finish_reason": "tool_calls"
        }
    ],
    "usage": {
        "prompt_tokens": 95,
        "completion_tokens": 6,
        "total_tokens": 101
    }
}
```

In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content 

```json 
{
  "message_appended": {
    "content": null,
    "refusal": null,
    "role": "assistant",
    "annotations": [],
    "audio": null,
    "function_call": null,
    "tool_calls": [
      {
        "id": "call_EzAvAif0eL9y5fClyKJIvKD4",
        "function": {
          "arguments": {
            "destination_city": "Canada"
          },
          "name": "get_ticket_price"
        },
        "type": "function"
      }
    ]
  },
  "response_appended": {
    "role": "tool",
    "content": {
      "destination_city": "Canada",
      "price": "BDT 25000"
    },
    "tool_call_id": "call_EzAvAif0eL9y5fClyKJIvKD4"
  }
}


```

In [9]:
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('destination_city')
    price = get_ticket_prices(city)
    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city,"price": price}),
        "tool_call_id": tool_call.id
    }
    return response, city

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


Ticket price for Canada called
message appended: ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_EzAvAif0eL9y5fClyKJIvKD4', function=Function(arguments='{"destination_city":"Canada"}', name='get_ticket_price'), type='function')])
response appended: {'role': 'tool', 'content': '{"destination_city": "Canada", "price": "BDT 25000"}', 'tool_call_id': 'call_EzAvAif0eL9y5fClyKJIvKD4'}
